## Data Cleaning
### Install Packages

In [4]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


### Import Packages

In [ ]:
import json, csv, re
from decimal import Decimal, InvalidOperation
from pathlib import Path
from typing import Dict, Iterable, List, Optional, Tuple
import pandas as pd

### Converting Review File

In [6]:
URL_RE = re.compile(r"(https?://|www\.)\S+", re.IGNORECASE)

def _looks_like_jsonl(p: Path, enc="utf-8") -> bool:
    try:
        head = p.read_text(encoding=enc, errors="replace")[:4096]
        return head.lstrip().startswith("{") and "\n" in head
    except Exception:
        return False

def _iter_jsonl(p: Path, enc="utf-8") -> Iterable[Dict]:
    with p.open("r", encoding=enc, errors="replace") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                obj = json.loads(line)
            except json.JSONDecodeError:
                continue
            if isinstance(obj, dict):
                yield obj

def _load_json_array(p: Path, enc="utf-8") -> List[Dict]:
    data = json.loads(p.read_text(encoding=enc, errors="replace"))
    if isinstance(data, dict):
        return [data]
    if isinstance(data, list):
        return [d for d in data if isinstance(d, dict)]
    raise ValueError("JSON must be an object or an array of objects.")

def _parse_ms_like(val) -> Optional[int]:
    if val is None or (isinstance(val, float) and pd.isna(val)):
        return None
    s = str(val).strip()
    if not s:
        return None
    try:
        if re.match(r"^[+-]?\d+(\.\d+)?[eE][+-]?\d+$", s):
            return int(Decimal(s))
    except InvalidOperation:
        pass
    try:
        if "." in s:
            return int(Decimal(s))
        return int(s)
    except Exception:
        return None

def _to_ymdt(val) -> Tuple[Optional[int], Optional[int], Optional[int], Optional[str]]:
    ms = _parse_ms_like(val)
    if ms is not None:
        dt = pd.to_datetime(ms, unit="ms", utc=True, errors="coerce")
    else:
        dt = pd.to_datetime(str(val), utc=True, errors="coerce")
    if pd.isna(dt):
        return None, None, None, None
    return int(dt.year), int(dt.month), int(dt.day), dt.strftime("%H:%M:%S")

def reviews_json_to_clean_csv(
    in_json: str,
    out_csv: str,
    drop_empty_text: bool = True,
    dedup: bool = True,
    keep_newlines: bool = False,
):
    in_path = Path(in_json)
    is_jsonl = _looks_like_jsonl(in_path)

    rows = list(_iter_jsonl(in_path)) if is_jsonl else _load_json_array(in_path)

    recs = []
    for r in rows:
        text = (r.get("text") or "")
        if not keep_newlines:
            text = str(text).replace("\r\n", " ").replace("\n", " ").replace("\r", " ")
        text = re.sub(r"[ \t]+", " ", str(text)).strip()

        y, m, d, t = _to_ymdt(r.get("time", r.get("timestamp")))
        has_pic = 0
        if "pics" in r and r["pics"] not in (None, "", "[]", "null"):
            has_pic = 1
        has_url = 1 if URL_RE.search(text) else 0

        recs.append({
            "user_id": r.get("user_id"),
            "gmap_id": r.get("gmap_id"),
            "rating": r.get("rating"),
            "year": y, "month": m, "day": d, "time": t,
            "has_picture": has_pic,
            "has_url": has_url,
            "text": text
        })

    df = pd.DataFrame(recs)

    if drop_empty_text:
        df = df[df["text"].astype(str).str.strip() != ""]

    if dedup:
        subset = ["text"]
        if "gmap_id" in df.columns:
            subset = ["gmap_id", "text"]
        df = df.drop_duplicates(subset=subset, keep="first")

    cols = ["user_id","gmap_id","rating","year","month","day","time","has_picture","has_url","text"]
    for c in cols:
        if c not in df.columns:
            df[c] = None
    df = df[cols].copy()

    df.to_csv(out_csv, index=False, encoding="utf-8-sig", quoting=csv.QUOTE_ALL, lineterminator="\n")
    return df


In [ ]:
review_clean = reviews_json_to_clean_csv(
    in_json="./data/review-Vermont_10.json",
    out_csv="./data/reviews_Vermont.csv",
    drop_empty_text=True,
    dedup=True
)
review_clean.head()


,user_id,gmap_id,rating,year,month,day,time,has_picture,has_url,text
0,118026874392842649478,0x89e02445cb9db457:0x37f42bff4edf7a43,5,2021,5,3,23:50:52,0,0,Always done right from wood stove to screens!
1,101532740754036204131,0x89e02445cb9db457:0x37f42bff4edf7a43,5,2020,1,29,14:59:06,0,0,A great company to work with. Their sales and ...
2,115404122636203550540,0x89e02445cb9db457:0x37f42bff4edf7a43,5,2020,11,12,15:46:14,0,0,Great place to do business with staff was grea...
3,104789336434407408181,0x89e02445cb9db457:0x37f42bff4edf7a43,5,2020,6,24,13:37:28,0,0,"Awesome Customer service, quick response, and ..."
4,108980665975608069965,0x89e02445cb9db457:0x37f42bff4edf7a43,5,2020,2,18,21:06:36,0,0,"If you need a top quality job, by a group of p..."


### Converting Metadata file

In [15]:
METADATA_COLS = [
    "name","address","gmap_id","description","category", "avg_rating","num_of_reviews","state","url"
]

def _to_cell(v):
    """Serialize lists/dicts to JSON; keep scalars; empty string for None."""
    if isinstance(v, (list, dict)):
        return json.dumps(v, ensure_ascii=False)
    return "" if v is None else v

def meta_json_to_clean_csv(
    in_json: str,
    out_csv: str,
    dedup: bool = True,
):
    in_path = Path(in_json)
    is_jsonl = _looks_like_jsonl(in_path)

    rows = list(_iter_jsonl(in_path)) if is_jsonl else _load_json_array(in_path)

    recs: List[Dict] = []
    for r in rows:
        recs.append({
            "name": r.get("name"),
            "address": r.get("address"),
            "gmap_id": r.get("gmap_id"),
            "description": r.get("description"),
            "category": _to_cell(r.get("category")),
            "avg_rating": r.get("avg_rating"),
            "num_of_reviews": r.get("num_of_reviews"),
            "state": r.get("state"),
            "url": r.get("url"),
        })

    df = pd.DataFrame(recs)

    for c in METADATA_COLS:
        if c not in df.columns:
            df[c] = ""
    df = df[METADATA_COLS].copy()

    if dedup and "gmap_id" in df.columns:
        df = df.drop_duplicates(subset=["gmap_id"], keep="first")

    df.to_csv(out_csv, index=False, encoding="utf-8-sig",
              quoting=csv.QUOTE_ALL, lineterminator="\n")
    return df

In [16]:
metadata_clean = meta_json_to_clean_csv(
    in_json="./data/meta-Vermont.json",
    out_csv="./data/meta_Vermont.csv",
    dedup=True
)
metadata_clean.head()


,name,address,gmap_id,description,category,avg_rating,num_of_reviews,state,url
0,Royal Group,"Royal Group, 150 Woodstock Ave, Rutland, VT 05701",0x89e02445cb9db457:0x37f42bff4edf7a43,None,"[""Security system supplier"", ""Fire protection ...",4.9,148,Closed ⋅ Opens 8AM,https://www.google.com/maps/place//data=!4m2!3...
1,Foxglove Farm and Forest,"Foxglove Farm and Forest, 777 Delorm Rd, Leice...",0x4cb549e8877cf0d7:0xe8f003e6d73392ae,None,"[""Indoor lodging"", ""Farm"", ""Gift shop""]",5.0,3,None,https://www.google.com/maps/place//data=!4m2!3...
2,Carr's Gifts,"Carr's Gifts, 21 Center St, Brandon, VT 05733",0x4cb54a301f3518f7:0x39af4eda1efb9117,None,"[""Gift shop""]",4.9,13,Closed ⋅ Opens 10AM Thu,https://www.google.com/maps/place//data=!4m2!3...
3,Midas,"Midas, 207 US Route 4 E, Woodstock Ave, Rutlan...",0x89e024446398691f:0x4011d3a0f8636036,None,"[""Auto repair shop"", ""Brake shop"", ""Muffler sh...",4.3,48,Closed ⋅ Opens 7:30AM Thu,https://www.google.com/maps/place//data=!4m2!3...
4,Keyser Energy,"Keyser Energy, 77 Grove St Suite G102, Rutland...",0x89e0247d160d7263:0x32e4f01896e33f3b,None,"[""Heating oil supplier"", ""Air conditioning con...",4.7,68,Closed ⋅ Opens 8AM Thu,https://www.google.com/maps/place//data=!4m2!3...


### Joining

In [18]:
def join_on_gmap_id(
    reviews_df: pd.DataFrame,
    meta_df: pd.DataFrame,
    out_csv: str = "reviews_meta.csv",
    keep_meta=("name", "category"),
    join: str = "inner",   
) -> pd.DataFrame:
    for name, df in (("reviews_df", reviews_df), ("meta_df", meta_df)):
        if "gmap_id" not in df.columns:
            raise ValueError(f"{name} must contain a 'gmap_id' column")

    r = reviews_df.copy()
    m = meta_df.copy()
    r["gmap_id"] = r["gmap_id"].astype(str).str.strip()
    m["gmap_id"] = m["gmap_id"].astype(str).str.strip()

    cols_meta = ["gmap_id"] + [c for c in keep_meta if c in m.columns]
    m = m[cols_meta].drop_duplicates(subset=["gmap_id"], keep="first")

    merged = r.merge(m, on="gmap_id", how=join, validate="m:1")

    review_first = [c for c in r.columns] 
    meta_tail = [c for c in keep_meta if c in merged.columns]
    ordered_cols = review_first + [c for c in meta_tail if c not in review_first]
    merged = merged[ordered_cols]

    merged.to_csv(out_csv, index=False, encoding="utf-8-sig",
                  quoting=csv.QUOTE_ALL, lineterminator="\n")
    return merged

In [19]:
final_df = join_on_gmap_id(
    reviews_df=review_clean,
    meta_df=metadata_clean,
    out_csv="./data/reviews_meta.csv",
    keep_meta=("name","category"),
    join="inner"
)

final_df.head()


,user_id,gmap_id,rating,year,month,day,time,has_picture,has_url,text,name,category
0,118026874392842649478,0x89e02445cb9db457:0x37f42bff4edf7a43,5,2021,5,3,23:50:52,0,0,Always done right from wood stove to screens!,Royal Group,"[""Security system supplier"", ""Fire protection ..."
1,101532740754036204131,0x89e02445cb9db457:0x37f42bff4edf7a43,5,2020,1,29,14:59:06,0,0,A great company to work with. Their sales and ...,Royal Group,"[""Security system supplier"", ""Fire protection ..."
2,115404122636203550540,0x89e02445cb9db457:0x37f42bff4edf7a43,5,2020,11,12,15:46:14,0,0,Great place to do business with staff was grea...,Royal Group,"[""Security system supplier"", ""Fire protection ..."
3,104789336434407408181,0x89e02445cb9db457:0x37f42bff4edf7a43,5,2020,6,24,13:37:28,0,0,"Awesome Customer service, quick response, and ...",Royal Group,"[""Security system supplier"", ""Fire protection ..."
4,108980665975608069965,0x89e02445cb9db457:0x37f42bff4edf7a43,5,2020,2,18,21:06:36,0,0,"If you need a top quality job, by a group of p...",Royal Group,"[""Security system supplier"", ""Fire protection ..."
